In [1]:
import warnings
warnings.filterwarnings(action='ignore')
%config Completer.use_jedi = False
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np

Instructions for updating:
non-resource variables are not supported in the long term


***
RNN(Recurrent_Neural _Network) - 순환 신경망
***
RNN은 순차적인 데이터를 입력받아 결과값을 도출하는 데 사용하는 딥러닝 모델로  
자연어 처리에 상당히 많이 사용되고 이전에 입력된 값들을 고려해서 현재 입력값의 출력값을 결정하는 딥러닝 모델이다.  

x는 입력값, y는 출력값, 활성화 함수(tanh)를 거친 값은 상태(출력)을 의미한다.  
네모 박스는 셀이라하며 셀 안에서 현재 셀의 입력값과 과거셀의 상태값을 사용해 현재 셀의 상태(출력)값을 계산한다.  
현재 셀의 상태값은 현재 셀의 출력값과 동일하며 다음 셀의 이전 상태값으로 사용된다.  

상태값을 결정하기 위해서는 다음 그림과 같이 두가지의 가중치가 존재한다.  
현재 셀의 상태값은 tanh(입력값 * W<sub>xh</sub> + 이전 셀의 상태값 * W<sub>hh</sub> + 편향값)으로 결정된다.  
가중치와 편향값은 최초 무작위로 부여하고 학습과정을 통해 가중치 및 편향값은 목적에 맞게 최적화된다.

<img src="./RNN.png" width="700px">

***
tensorflow - RNN 코드 구현
***

In [2]:
inputs = np.array([[[1, 2]]]) #입력데이터
#print(inputs) # [[[1 2]]]
#print(inputs.shape) # (1, 1, 2)
#print(inputs.shape[0]) # 1
tf.set_random_seed(35)

tf_inputs = tf.constant(inputs, dtype=tf.float32)
sess = tf.Session()
print('입력데이터 -> {}'.format(sess.run(tf_inputs)))


#BasicRNNCell() 객체를 생성할 때 생성자의 인수 num_units로 RNN 셀의 갯수를 지정해서 RNN 객체를 생성한다.
rnn_cell = tf.nn.rnn_cell.BasicRNNCell(num_units=4) #위 그림의 네모박스
print( 'RNN 셀의 갯수 -> {}'.format(rnn_cell.state_size) )
print('*' * 100)

# dynamic_rnn() 메소드는 입력값과 RNN셀을 받아서 실행 결과(출력값, 상태값)를 리턴한다.
# dynamic_rnn(cell, dtype, inputs)
# cell : 실행할 RNN 객체
# dtype : 데이터 타입
# inputs : RNN 셀에 입력되는 데이터
outputs, state = tf.nn.dynamic_rnn(cell=rnn_cell, dtype=tf.float32, inputs=tf_inputs)
print('*' * 100)
print( '출력값 -> {}'.format(outputs) )
print( '상태값 -> {}'.format(state) )
print('*' * 100)
print( '가중치 갯수와 편향값의 개수' )
for v in tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES) :
     print(v)

입력데이터 -> [[[1. 2.]]]
RNN 셀의 갯수 -> 4
****************************************************************************************************
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
****************************************************************************************************
출력값 -> Tensor("rnn/transpose_1:0", shape=(1, 1, 4), dtype=float32)
상태값 -> Tensor("rnn/while/Exit_3:0", shape=(1, 4), dtype=float32)
****************************************************************************************************
가중치 갯수와 편향값의 개수
<tf.Variable 'rnn/basic_rnn_cell/kernel:0' shape=(6, 4) dtype=float32_ref>
<tf.Variable 'rnn/basic_rnn_cell/bias:0' shape=(4,) dtype=float32_ref>


In [3]:
variable_name = [vari.name for vari in tf.trainable_variables()]
print(variable_name)
with tf.Session() as sess:
     sess.run(tf.global_variables_initializer())
     outputs_, state_ = sess.run([outputs, state])
     # 출력값과 상태값은 같은 값이고, 상태값이 다음 RNN 셀로 전달된다.
     print( '출력값 -> {}'.format(outputs_) )
     print( '최종 상태값 -> {}'.format(state_) )
     
     values = sess.run(variable_name)
     print('가중치 -> ', variable_name[0], '\n', values[0], sep='')
     print('편향값 -> ', variable_name[1], '\n', values[1], sep='')

['rnn/basic_rnn_cell/kernel:0', 'rnn/basic_rnn_cell/bias:0']
출력값 -> [[[-0.7551901  -0.10510498 -0.40611452  0.17652836]]]
최종 상태값 -> [[-0.7551901  -0.10510498 -0.40611452  0.17652836]]
가중치 -> rnn/basic_rnn_cell/kernel:0
[[ 0.39129794  0.27990448 -0.03417063 -0.13734835]
 [-0.6881115  -0.19269955 -0.19838947  0.15787274]
 [-0.3503854  -0.74573535  0.42230952  0.06174737]
 [-0.07638937  0.737661   -0.0250259   0.22168279]
 [ 0.6998607   0.48419607  0.55390215  0.55799115]
 [ 0.26617455  0.20661819 -0.65159154 -0.75656563]]
편향값 -> rnn/basic_rnn_cell/bias:0
[0. 0. 0. 0.]


***
RNN 셀은 출력값과 상태값이 동일하다.  
RNN 셀이 1개일 경우 가중치는 3개의 행과 1개의 열을 가지고, 편향값은 1개가 필요하다.  
RNN 셀이 2개일 경우 가중치는 4개의 행과 2개의 열을 가지고, 편향값은 2개가 필요하다.  
RNN 셀이 3개일 경우 가중치는 5개의 행과 3개의 열을 가지고, 편향값은 3개가 필요하다.  
RNN 셀이 4개일 경우 가중치는 6개의 행과 4개의 열을 가지고, 편향값은 4개가 필요하다.  
...  
RNN 셀이 N개일 경우 가중치는 N+2개의 행과 N개의 열을 가지고, 편향값은 N개가 필요하다.  
***